In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
using Test
using ForwardDiff
using SpecialFunctions
using Polynomials
using BenchmarkTools
using TransportMap
using QuadGK
using FastGaussQuadrature
using AdaptiveTransportMap: derivative, vander
using DiffResults
using QuadGK

### Basis

In [3]:
Nx = 6
B = CstProHermite(7)
MB = MultiBasis(B, Nx)

MultiBasis{9,6}(Basis of 9 functions:
ProPolyHermite{0}(ImmutablePolynomial(1.0), false)
ProHermite{0}(ProPolyHermite{0}(ImmutablePolynomial(1.0), false), false)
ProHermite{1}(ProPolyHermite{1}(ImmutablePolynomial(1.0*x), false), false)
ProHermite{2}(ProPolyHermite{2}(ImmutablePolynomial(-1.0 + 1.0*x^2), false), false)
ProHermite{3}(ProPolyHermite{3}(ImmutablePolynomial(-3.0*x + 1.0*x^3), false), false)
ProHermite{4}(ProPolyHermite{4}(ImmutablePolynomial(3.0 - 6.0*x^2 + 1.0*x^4), false), false)
ProHermite{5}(ProPolyHermite{5}(ImmutablePolynomial(15.0*x - 10.0*x^3 + 1.0*x^5), false), false)
ProHermite{6}(ProPolyHermite{6}(ImmutablePolynomial(-15.0 + 45.0*x^2 - 15.0*x^4 + 1.0*x^6), false), false)
ProHermite{7}(ProPolyHermite{7}(ImmutablePolynomial(-105.0*x + 105.0*x^3 - 21.0*x^5 + 1.0*x^7), false), false)
)

f (generic function with 1 method)

In [4]:
@btime ForwardDiff.gradient(x->B[5](x[1]),[5.0])

  1.867 μs (9 allocations: 480 bytes)


1-element Array{Float64,1}:
 -0.39188218965422494

In [5]:
@btime ForwardDiff.derivative(x->B[5](x),5.0)

  107.739 ns (6 allocations: 176 bytes)


-0.39188218965422494

In [6]:
function timing()
    f(x) = (x^5-4*x^3+1)*exp(-x^2/2)  
    @btime ForwardDiff.gradient(x->f(x[1]),[5.0])
    @btime ForwardDiff.derivative(x->f(x),5.0)
end

timing (generic function with 1 method)

In [51]:
FamilyPhyHermite[5].P

PhyHermite{4}(PhyPolyHermite{4}(ImmutablePolynomial(12.0 - 48.0*x^2 + 16.0*x^4), false), false)

In [8]:
@btime quadgk(x->FamilyPhyHermite[5](x), 0, 1.0)

  2.024 μs (137 allocations: 2.34 KiB)


(0.5630021473036511, 1.4497070210950369e-11)

In [9]:
Nx = 10
Nψ = 8
B = CstProHermite(4)
MB = MultiBasis(B, Nx)
f = ExpandedFunction([MultiFunction(MB, rand(1:6,10)) for i=1:5], randn(5));

In [10]:
result = DiffResults.HessianResult(randn(10));
@time result = ForwardDiff.gradient!(result, f, randn(10));

MethodError: MethodError: no method matching getindex(::MultiBasis{6,10}, ::Int64)

In [11]:
first()

MethodError: MethodError: no method matching first()
Closest candidates are:
  first(!Matched::Tuple{}) at tuple.jl:95
  first(!Matched::BitSet) at bitset.jl:356
  first(!Matched::Core.Compiler.StmtRange) at show.jl:1848
  ...

In [12]:
function timing2()
    @btime x = randn(100,20)
    @btime begin 
        x = randn(20,200)
        for i=1:200
        x[10,i]
        end
    end
end

timing2 (generic function with 1 method)

In [13]:
timing2()

  10.325 μs (1 allocation: 15.75 KiB)
  20.985 μs (2 allocations: 31.33 KiB)


In [14]:
function timing()
    Nx = 10
    Nψ = 8
    B = CstProHermite(4)
    MB = MultiBasis(B, Nx);
    f = ExpandedFunction([MultiFunction(MB, rand(1:6,10)) for i=1:5], randn(5));
    [f(randn(10)) for i=1:200]
end

timing (generic function with 1 method)

In [15]:
@btime timing()

MethodError: MethodError: no method matching getindex(::MultiBasis{6,10}, ::Int64)

In [16]:
@time [f(randn(10)) for i=1:200]

MethodError: MethodError: no method matching getindex(::MultiBasis{6,10}, ::Int64)

In [17]:
@btime ForwardDiff.gradient(f, randn(10))

MethodError: MethodError: no method matching getindex(::MultiBasis{6,10}, ::Int64)

In [18]:
function timing() 
    f = MultiFunction(MB, rand(1:6,10));
    result = DiffResults.HessianResult(randn(10));
    result = ForwardDiff.hessian!(result, f, randn(10));
end

timing (generic function with 1 method)

In [19]:
function timing2() 
    f = MultiFunction(MB, rand(1:6,10));
    x = randn(10)
    f(x)
    ForwardDiff.gradient(f, x);
    ForwardDiff.hessian(f, x);
end

timing2 (generic function with 1 method)

In [20]:
@btime timing()

MethodError: MethodError: no method matching getindex(::MultiBasis{6,10}, ::Int64)

In [21]:
@btime timing2()

MethodError: MethodError: no method matching getindex(::MultiBasis{6,10}, ::Int64)

### Hermite functions

In [68]:
FamilyScaledPhyHermite[5]

PhyHermite{4}(PhyPolyHermite{4}(ImmutablePolynomial(0.4599685791773267 - 1.8398743167093068*x^2 + 0.6132914389031022*x^4), true), true)

In [61]:
integral, err = quadgk(x-> (12.0 -48*x^2 + 16*x^4)*exp(-x^2/2), 0.0, 1.0, rtol=1e-12)

(0.5630021473036511, 8.881784197001252e-16)

In [102]:
@testset "Verify integration of Probabilistic hermite functions" begin
    Nx = 10
    x = randn(10)

    # Unscaled
    dV = vander(ProHermite(8; scaled = false), -1, x)

    for i=0:8
        for j=1:Nx
            @test abs(dV[j,i+1] - quadgk(y-> ProHermite(i)(y), 0.0, x[j], rtol=1e-12)[1])<1e-8
        end
    end

    # Scaled
    dV = vander(ProHermite(8; scaled = true), -1, x)

    for i=0:8
        for j=1:Nx
            @test abs(dV[j,i+1] - quadgk(y-> ProHermite(i; scaled=true)(y), 0.0, x[j], rtol=1e-12)[1])<1e-8
        end
    end

end

dF = [-0.740475987583314, -0.2118041248390954, -0.723737006762011, 0.16332198675338616, -1.1349653390330037, -0.45474318136403646, 0.14037770514365783, 0.2407111754716413, -1.1110238905632188, 0.29979314335595375]
1 / sqrt(2 ^ m) = 1.0
dF = [-0.740475987583314, -0.2118041248390954, -0.723737006762011, 0.16332198675338616, -1.1349653390330037, -0.45474318136403646, 0.14037770514365783, 0.2407111754716413, -1.1110238905632188, 0.29979314335595375]
dF = [-1.0471911842519344, -0.29953626591401283, -1.0235186905541445, 0.2309721687003577, -1.6050833752838518, -0.6431039744817085, 0.1985240544689723, 0.3404170089667651, -1.5712250541550254, 0.42397152924045134]
dF = [0.5772887540590417, 0.04503047777933733, 0.5500498481714482, 0.02673373578416305, 1.506116803232493, 0.21053938093542235, 0.019738410060434308, 0.058225496753504524, 1.4282398231145264, 0.0905636111977175]
1 / sqrt(2 ^ m) = 0.7071067811865475
dF = [0.4082047926978814, 0.03184135619783957, 0.38894397763266186, 0.01890360585943115

Test Summary:                                         | Pass  Total
Verify integration of Probabilistic hermite functions |  180    180


Test.DefaultTestSet("Verify integration of Probabilistic hermite functions", Any[], 180, false)